In [1]:
import pandas as pd


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
training=pd.read_csv('/content/drive/MyDrive/QC/training (1).csv')

In [5]:
test=pd.read_csv('/content/drive/MyDrive/QC/test.csv')

In [6]:
training_new=pd.read_csv('/content/drive/MyDrive/QC/training_new.csv')

In [7]:
test_new=pd.read_csv('/content/drive/MyDrive/QC/test_new.csv')

data processing:

In [8]:
import lightgbm as lgb
from sklearn.model_selection import training_test_split
from sklearn.metrics import r2_score
import pandas as pd

features_core=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']







In [8]:
##combining features
X_combiningd=pd.concat([training[features_core],training_new[new_features]],axis=1)

##split 80% training,20% val for both targets
X_training,X_val,y1_training,y1_val=training_test_split(X_combiningd,training['Y1'],test_size=0.2,random_state=1,shuffle=True)
_,_,y2_training,y2_val=training_test_split(X_combiningd,training['Y2'],test_size=0.2,random_state=1,shuffle=True)

**Tune Y1 without K-Fold:**



In [18]:
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.metrics import r2_score

##defi hyperparameters in one dict
params_y1={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.290108925533801, #27-done
    'learning_rate': 0.03346736377247299, #33-done
    'max_depth': 8,#done
    'n_estimators': 800,#800 - done
    'num_leaves': 54,#54-done
    'lambda_l1': 0.8358897454113226,#83 -done
    'lambda_l2': 0.23465766293184155,#23 -done
    'verbosity': -1,
    'random_state': 1
}

##training model with early stopping on val set
model_y1=lgb.LGBMRegressor(**params_y1)

model_y1.fit(
    X_training,y1_training,
    eval_set=[(X_val,y1_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50)],
    #verbose=False
)

##predicting on validation set
y1_val_pred=model_y1.predicting(X_val)
val_r2_y1=r2_score(y1_val,y1_val_pred)
print(f"Y1 Validation R2: {val_r2_y1}")

##Log results to CSV and print
import pandas as pd
log_path='/content/drive/MyDrive/QC/y1_res.csv'

##import pandas as pd
from pandas.errors import EmptyDataError

try:
    log_df=pd.read_csv('/content/drive/MyDrive/QC/y1_res.csv')
except (FileNotFoundError,EmptyDataError):
    ##Handle file not existing or empty
    log_df=pd.DataFrame()




entry=params_y1.copy()
entry['validation_r2']=val_r2_y1

log_df=pd.concat([log_df,pd.DataFrame([entry])],ignore_index=True)



log_df.to_csv(log_path,index=False)

print("Logged current parameters and validation R2 to y1_res.csv")
print(log_df.tail(5))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[499]	valid_0's rmse: 0.45012
Y1 Validation R2: 0.7775851157118706
Logged current parameters and validation R2 to y1_res.csv
     objective metric  feature_fraction  learning_rate  max_depth  \
74  regression   rmse          0.290109       0.033467          8   
75  regression   rmse          0.650000       0.010000         10   
76  regression   rmse          0.290109       0.330000          8   
77  regression   rmse          0.290109       0.033467          8   
78  regression   rmse          0.290109       0.033467          8   

    n_estimators  num_leaves  lambda_l1  lambda_l2  verbosity  random_state  \
74           800          54    0.83589   0.234658         -1            42   
75           700          60    0.83589   0.234658         -1            42   
76           800          54    0.83589   0.234658         -1            42   
77           800          54    0.83589   0.2346

**Tune Y2 without K-Fold:**

In [12]:
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.metrics import r2_score
import pandas as pd
from pandas.errors import EmptyDataError

##defi hyperparameters for Y2
params_y2={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.4,#0.5 -done
    'learning_rate': 0.05, #0.05 -done
    'max_depth': 7,#done -7
    'n_estimators': 800,#done -800
    'num_leaves': 47,#41 - done
    #'lambda_l1' : 0.9358897454113226,#none
    #'lambda_l2': 0.93465766293184155,#none
    'verbosity': -1,
    'random_state': 1
}

##traininging with early stopping
model_y2=lgb.LGBMRegressor(**params_y2)

model_y2.fit(
    X_training,y2_training,
    eval_set=[(X_val,y2_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50)],
    ##verbose=False
)

##predicting on validation set
y2_val_pred=model_y2.predicting(X_val)
val_r2_y2=r2_score(y2_val,y2_val_pred)
print(f"Y2 Validation R2: {val_r2_y2}")

##log the results to CSV and print
log_path_y2='/content/drive/MyDrive/QC/y2_res.csv'

try:
    log_df_y2=pd.read_csv(log_path_y2)
except (FileNotFoundError,EmptyDataError):
    log_df_y2=pd.DataFrame()

entry_y2=params_y2.copy()
entry_y2['validation_r2']=val_r2_y2

log_df_y2=pd.concat([log_df_y2,pd.DataFrame([entry_y2])],ignore_index=True)

log_df_y2.to_csv(log_path_y2,index=False)

print("Logged current parameters and validation R2 to y2_res.csv")
print(log_df_y2.tail(5))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[592]	valid_0's rmse: 0.418602
Y2 Validation R2: 0.8033225693098163
Logged current parameters and validation R2 to y2_res.csv
     objective metric  feature_fraction  learning_rate  max_depth  \
69  regression   rmse               0.5           0.05          7   
70  regression   rmse               0.5           0.05          7   
71  regression   rmse               0.5           0.05          7   
72  regression   rmse               0.5           0.05          7   
73  regression   rmse               0.4           0.05          7   

    n_estimators  num_leaves  verbosity  random_state  validation_r2  \
69           800          47         -1            42       0.801339   
70           800          47         -1            42       0.802802   
71           800          47         -1            42       0.804228   
72           800          47         -1            42       0.806805   
73 

**SUBMISSION7**

In [ ]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.2901089255338042,
    learning_rate=0.03346736377247299,
    max_depth=8,
    n_estimators=800,
    num_leaves=54,
    lambda_l1=0.8358897454113226,
    lambda_l2=0.23465766293184155,
    verbosity=-1,
    random_state=1
)

##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2 with given best params
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.7,
    learning_rate=0.05,
    max_depth=7,
    n_estimators=500,
    num_leaves=31,
    verbosity=-1,
    random_state=1
)


##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_best_params.csv',index=False)
print("Submission saved as submission_best_params.csv")


Submission saved as submission_best_params.csv


**`SUBMISSION 8`**

In [ ]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.2901089255338042,
    learning_rate=0.03346736377247299,
    max_depth=8,
    n_estimators=800,
    num_leaves=54,
    lambda_l1=0.8358897454113226,
    lambda_l2=0.23465766293184155,
    verbosity=-1,
    random_state=1
)

##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2 with updated params
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.5,
    learning_rate=0.05,
    max_depth=7,
    n_estimators=800,
    num_leaves=47,
    verbosity=-1,
    random_state=1
)

##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission7_9pm_21.csv',index=False)
print("Submission saved as submission7.csv")


Submission saved as submission7.csv


**SUBMISSION9**

In [10]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.65,
    learning_rate=0.01,
    max_depth=10,
    n_estimators=700,
    num_leaves=60,
    lambda_l1=0.8358897454113226,
    lambda_l2=0.23465766293184155,
    verbosity=-1,
    random_state=1
)

##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2 with updated params as
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.65,
    learning_rate=0.03,
    max_depth=8,
    n_estimators=700,
    num_leaves=32,
    verbosity=-1,
    random_state=1
)

##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_check_with1st.csv',index=False)
print("Submission saved as submission7.csv")


Submission saved as submission7.csv


**CHECK THE R score of SUBMISSION 9 on 20% validation set:**

**Y1:**

In [11]:
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.metrics import r2_score

##defi hyperparameters in one dictionary
params_y1={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.2901089255338042, #27-done
    'learning_rate': 0.03346736377247299, #33-done
    'max_depth': 8,#8 - done
    'n_estimators': 800,#800 - done
    'num_leaves': 54,#54-done
    'lambda_l1': 0.8358897454113226,#83 -done
    'lambda_l2': 0.23465766293184155,#23 -done
    'verbosity': -1,
    'random_state': 1
}

##training model with early stopping on val set
model_y1=lgb.LGBMRegressor(**params_y1)

model_y1.fit(
    X_training,y1_training,
    eval_set=[(X_val,y1_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50)],
    #verbose=False
)

##predicting on validation set
y1_val_pred=model_y1.predicting(X_val)
val_r2_y1=r2_score(y1_val,y1_val_pred)
print(f"Y1 Validation R2: {val_r2_y1}")

##log results to CSV and print
import pandas as pd
log_path='/content/drive/MyDrive/QC/y1_res.csv'

##import pandas as pd
from pandas.errors import EmptyDataError

try:
    log_df=pd.read_csv('/content/drive/MyDrive/QC/y1_res.csv')
except (FileNotFoundError,EmptyDataError):
    ##Handle file not existing or empty
    log_df=pd.DataFrame()




entry=params_y1.copy()
entry['validation_r2']=val_r2_y1

log_df=pd.concat([log_df,pd.DataFrame([entry])],ignore_index=True)



log_df.to_csv(log_path,index=False)

print("Logged current parameters and validation R2 to y1_res.csv")
print(log_df.tail(5))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[499]	valid_0's rmse: 0.45012
Y1 Validation R2: 0.7775851157118706
Logged current parameters and validation R2 to y1_res.csv
     objective metric  feature_fraction  learning_rate  max_depth  \
73  regression   rmse          0.290109       0.033467          8   
74  regression   rmse          0.290109       0.033467          8   
75  regression   rmse          0.650000       0.010000         10   
76  regression   rmse          0.290109       0.330000          8   
77  regression   rmse          0.290109       0.033467          8   

    n_estimators  num_leaves  lambda_l1  lambda_l2  verbosity  random_state  \
73           800          54    0.83589   0.934658         -1            42   
74           800          54    0.83589   0.234658         -1            42   
75           700          60    0.83589   0.234658         -1            42   
76           800          54    0.83589   0.2346

**Y2:**

In [12]:
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.metrics import r2_score
import pandas as pd
from pandas.errors import EmptyDataError

##defi hyperparameters for Y2
params_y2={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.65,#0.5 -done
    'learning_rate': 0.03, #0.05 -done
    'max_depth': 8,#done -7
    'n_estimators': 700,#done -800
    'num_leaves': 32,#41 - done
    #'lambda_l1' : 0.9358897454113226,#none
    #'lambda_l2': 0.93465766293184155,#none
    'verbosity': -1,
    'random_state': 1
}

##traininging with early stopping
model_y2=lgb.LGBMRegressor(**params_y2)

model_y2.fit(
    X_training,y2_training,
    eval_set=[(X_val,y2_val)],
    eval_metric='rmse',
    callbacks=[early_stopping(stopping_rounds=50)],
    ##verbose=False
)

##predicting on validation set
y2_val_pred=model_y2.predicting(X_val)
val_r2_y2=r2_score(y2_val,y2_val_pred)
print(f"Y2 Validation R2: {val_r2_y2}")

##Log results to CSV and print
log_path_y2='/content/drive/MyDrive/QC/y2_res.csv'

try:
    log_df_y2=pd.read_csv(log_path_y2)
except (FileNotFoundError,EmptyDataError):
    log_df_y2=pd.DataFrame()


entry_y2=params_y2.copy()
entry_y2['validation_r2']=val_r2_y2

log_df_y2=pd.concat([log_df_y2,pd.DataFrame([entry_y2])],ignore_index=True)

log_df_y2.to_csv(log_path_y2,index=False)

print("Logged current parameters and validation R2 to y2_res.csv")
print(log_df_y2.tail(5))


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[700]	valid_0's rmse: 0.422082
Y2 Validation R2: 0.8000387954671213
Logged current parameters and validation R2 to y2_res.csv
     objective metric  feature_fraction  learning_rate  max_depth  \
70  regression   rmse              0.50           0.05          7   
71  regression   rmse              0.50           0.05          7   
72  regression   rmse              0.50           0.05          7   
73  regression   rmse              0.40           0.05          7   
74  regression   rmse              0.65           0.03          8   

    n_estimators  num_leaves  verbosity  random_state  validation_r2  \
70           800          47         -1            42       0.802802   
71           800          47         -1            42       0.804228   
72           800          47         -1            42       0.806805   
73           800          47         -1            42       0.

**Conclusion:**

*   The submission 7 and 8 overfitted on 20% validation data and thus they gave less R2 score compared to submission 9
*   When submission 9's parameters are checked with that 20% validation data,I got the R2 score less than R2 score i got with submission 7 and 8,that means,i overfitted submission 7 and 8 on this 20% validation data.
*   This 20% validation is taken with random split,but seed 1,and maintain same across all the notebook,or all the submission i made





**Next Steps:**
*   I will check how much score does the best submission parameters - submission 9's parameters give on the 3 cross validation - the average of all 3 scores

*   Then I will tune the hyperparameters starting from the submission 9's parameters ,such that they give better 3 fold validations score compared to the submission 9's 3-fold validation score
*   List item




**MANUAL TUNING ON 3-FOLD FOR BEST PARAMETERS:**

**Y1:**

In [55]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from pandas.errors import EmptyDataError

##defi features and target for Y1
features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']
X_y1=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_y1=training['Y1']

#optuna_search_parameters(with op)(without cross fold):
#Final : 0.754894-    Submitted - 0.5975
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.5524478320022036,
##    'learning_rate': 0.019553385840773797,
##    'max_depth': 9,
##    'n_estimators': 1500,
##    'num_leaves': 48,
##    'verbosity': -1,
##    'random_state': 1,
##     'lambda_l1': 0.9263413222927331,
##     'lambda_l2': 0.5416180534221455,
##     'min_child_samples': 14,
##     'bagging_fraction': 0.7054945561615795,
##     'bagging_freq': 6
##}

#grid_search_parameters(with cross fold):
###Final: 0.752910 - submitted - 0.710 - Ankur Submission ( i submitted with lamdas)
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.65,
##    'learning_rate': 0.01,
##    'max_depth': 10,
##    'n_estimators': 700,
##    'num_leaves': 60,
##    #'lambda_l1': 0.8358897454113226,
##    #'lambda_l2': 0.23465766293184155,
##    'verbosity': -1,
##    'random_state': 1
##}


###########################################################################################################################################
#optuna parameters(no cross fold):
#Final: 0.753753 - not submitted (y1_y2 file) - next this one - finetune and submit
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.5701089255338042,
##    'learning_rate': 0.03346736377247299,
##    'max_depth': 10,
##    'n_estimators': 800,
##    'num_leaves': 54,
##    'lambda_l1': 0.8458897454113226,#53 - #84
##    'lambda_l2': 0.43465766293184155,#43
##    'verbosity': -1,
##    'random_state': 1
##}

#passive/normal grid search(no cross fold) manual tuning:
###Final:  0.747115 - submitted - submission 7 and 8
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.2901089255338042,
##    'learning_rate': 0.03346736377247299,
##    'max_depth': 8,
##    'n_estimators': 800,
##    'num_leaves': 54,
##    'lambda_l1': 0.8358897454113226,#83 -done
##    'lambda_l2': 0.23465766293184155,#23 -done
##    'verbosity': -1,
##    'random_state': 1
##}

#grid_search_parameters(with cross fold)-copied from there:
#Final: 0.753379 - submitted 9
params={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.45,
    'learning_rate': 0.01,
    'max_depth': 10,
    'n_estimators': 700,
    'num_leaves': 60,
    'lambda_l1': 0.735,#0.9358897454113226 #73 #8897454113226
    'lambda_l2': 0.234,#0.23465766293184155 #23 #
    'verbosity': -1,
    'random_state': 1
}

##preparing the 3-fold cross validation (shuffle=False for time-series like behavior)
kf=KFold(n_splits=3,shuffle=False)

r2_scores=[]

for fold,(training_idx,val_idx) in enumerate(kf.split(X_y1)):
    print(f"traininging fold {fold+1}")
    X_training_fold,X_val_fold=X_y1.iloc[training_idx],X_y1.iloc[val_idx]
    y_training_fold,y_val_fold=y_y1.iloc[training_idx],y_y1.iloc[val_idx]

    model=lgb.LGBMRegressor(**params)

    model.fit(
        X_training_fold,
        y_training_fold,
        eval_set=[(X_val_fold,y_val_fold)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
        #verbose=False
    )

    y_val_pred=model.predicting(X_val_fold)
    fold_r2=r2_score(y_val_fold,y_val_pred)
    print(f"Fold {fold+1} R2: {fold_r2}")
    r2_scores.append(fold_r2)

mean_r2=sum(r2_scores) / len(r2_scores)
print(f"Mean R2 after 3-fold CV: {mean_r2}")

##Log mean R2 and params to CSV
log_path='/content/drive/MyDrive/QC/y1_cv_log.csv'

try:
    log_df=pd.read_csv(log_path)
except (FileNotFoundError,EmptyDataError):
    log_df=pd.DataFrame()

##preparing the dict with param values & score
log_entry=params.copy()
log_entry['mean_cv_r2']=mean_r2

log_df=pd.concat([log_df,pd.DataFrame([log_entry])],ignore_index=True)
log_df.to_csv(log_path,index=False)

print(f"Logged CV results to {log_path}")
print(log_df.tail())  ##Latest entries in log


Training fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[348]	valid_0's rmse: 0.35533
Fold 1 R2: 0.7819649697007693
Training fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[273]	valid_0's rmse: 0.470236
Fold 2 R2: 0.7477670493733117
Training fold 3
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[645]	valid_0's rmse: 0.605082
Fold 3 R2: 0.7328815620863225
Mean R2 after 3-fold CV: 0.7542045270534677
Logged CV results to /content/drive/MyDrive/QC/y1_cv_log.csv
     objective metric  feature_fraction  learning_rate  max_depth  \
77  regression   rmse              0.55           0.01         10   
78  regression   rmse              0.45           0.01         10   
79  regression   rmse              0.45           0.01         10   
80  regression   rmse              0.65           0.01          9   
81  regression   rmse           

**Y2:**

In [51]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from pandas.errors import EmptyDataError

##defi features and target for Y2,using old+new features
features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']
X_y2=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_y2=training['Y2']


#passive/normal grid search(no cross fold) manual tuning:
#Final : 0.648659    Submitted - Submission 8
#after lamda : 0.655779
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.5,
##    'learning_rate': 0.05,
##    'max_depth': 7,
##    'n_estimators': 800,
##    'num_leaves': 47,
##    'verbosity': -1,
##    'random_state': 1,
##    'lambda_l1': 0.9213114405513384,#92
##    'lambda_l2': 0.4520962952151522  #45
##}

#passive/normal grid search(no cross fold) no manual tuning:
#Final : 0.616942       Submitted - Submission 7
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.7,
##    'learning_rate': 0.05,
##    'max_depth': 7,
##    'n_estimators': 500,
##    'num_leaves': 31,
##    'verbosity': -1,
##    'random_state': 1
##}

#grid_search_parameters(with cross fold)-parameters copied: --used for deprecated
#Final : 0.628903          Submitted - Submission 9
params={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.45,
    'learning_rate': 0.03,
    'max_depth': 9,
    'n_estimators': 700,#700
    'num_leaves': 92,#32-#72
    'verbosity': -1,
    'random_state': 1,
    'lambda_l1': 0.9213114405513384,
    'lambda_l2': 0.6520962952151522
}

##########################################################################################################################################

#optuna_search_parameters(with op)(without cross fold):
#Final : 0.635082    Submitted - 0.5975 score on leaderboard
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.6086455125133671,
##    'learning_rate': 0.024435377516833785,
##    'max_depth': 9,
##    'n_estimators': 1500,
##    'num_leaves': 43,
##    'verbosity': -1,
##    'random_state': 1,
##    'lambda_l1': 0.6213114405513384,
##    'lambda_l2': 0.6520962952151522,
##    'min_child_samples': 13,
##    'bagging_fraction': 0.9278603731265997,
##    'bagging_freq': 7
##}





##preparing the 3-fold cross validation (shuffle=False)
kf=KFold(n_splits=3,shuffle=False)

r2_scores=[]

for fold,(training_idx,val_idx) in enumerate(kf.split(X_y2)):
    print(f"traininging fold {fold+1}")
    X_training_fold,X_val_fold=X_y2.iloc[training_idx],X_y2.iloc[val_idx]
    y_training_fold,y_val_fold=y_y2.iloc[training_idx],y_y2.iloc[val_idx]

    model=lgb.LGBMRegressor(**params)

    model.fit(
        X_training_fold,
        y_training_fold,
        eval_set=[(X_val_fold,y_val_fold)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
        #verbose=False
    )

    y_val_pred=model.predicting(X_val_fold)
    fold_r2=r2_score(y_val_fold,y_val_pred)
    print(f"Fold {fold+1} R2: {fold_r2}")
    r2_scores.append(fold_r2)

mean_r2=sum(r2_scores) / len(r2_scores)
print(f"Mean R2 after 3-fold CV for Y2: {mean_r2}")

##Log mean R2 and params - CSV
log_path='/content/drive/MyDrive/QC/y2_cv_log.csv'

try:
    log_df=pd.read_csv(log_path)
except (FileNotFoundError,EmptyDataError):
    log_df=pd.DataFrame()

##preparing the dict with param values & score
log_entry=params.copy()
log_entry['mean_cv_r2']=mean_r2

log_df=pd.concat([log_df,pd.DataFrame([log_entry])],ignore_index=True)
log_df.to_csv(log_path,index=False)

print(f"Logged CV results to {log_path}")
print(log_df.tail())  ##Latest entries in log


Training fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[104]	valid_0's rmse: 0.51172
Fold 1 R2: 0.6637336794201056
Training fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[47]	valid_0's rmse: 0.443454
Fold 2 R2: 0.6657167581597498
Training fold 3
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[205]	valid_0's rmse: 0.643169
Fold 3 R2: 0.6504824728996081
Mean R2 after 3-fold CV for Y2: 0.6599776368264879
Logged CV results to /content/drive/MyDrive/QC/y2_cv_log.csv
      objective metric  feature_fraction  learning_rate  max_depth  \
102  regression   rmse              0.55           0.03          9   
103  regression   rmse              0.45           0.03          9   
104  regression   rmse              0.40           0.03          9   
105  regression   rmse              0.42           0.03          9   
106  regression   rms

In [52]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from pandas.errors import EmptyDataError

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']
X_y2=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_y2=training['Y2']

base_params={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.45,
    'max_depth': 9,
    'n_estimators': 700,
    'num_leaves': 92,
    'verbosity': -1,
    'random_state': 1,
    'lambda_l1': 0.9213114405513384,
    'lambda_l2': 0.6520962952151522
}

kf=KFold(n_splits=3,shuffle=False)
log_path='/content/drive/MyDrive/QC/y2_cv_log.csv'

try:
    log_df=pd.read_csv(log_path)
except (FileNotFoundError,EmptyDataError):
    log_df=pd.DataFrame()

results=[]

for lr in [0.005,0.01,0.02,0.03,0.04,0.05,0.07,0.1]:
    params=base_params.copy()
    params['learning_rate']=lr
    r2_scores=[]
    for training_idx,val_idx in kf.split(X_y2):
        X_training_fold,X_val_fold=X_y2.iloc[training_idx],X_y2.iloc[val_idx]
        y_training_fold,y_val_fold=y_y2.iloc[training_idx],y_y2.iloc[val_idx]
        model=lgb.LGBMRegressor(**params)
        model.fit(
            X_training_fold,
            y_training_fold,
            eval_set=[(X_val_fold,y_val_fold)],
            eval_metric='rmse',
            callbacks=[lgb.early_stopping(stopping_rounds=50)],
        )
        y_val_pred=model.predicting(X_val_fold)
        r2_scores.append(r2_score(y_val_fold,y_val_pred))
    mean_r2=sum(r2_scores) / len(r2_scores)
    log_entry=params.copy()
    log_entry['mean_cv_r2']=mean_r2
    results.append(log_entry)

log_df=pd.concat([log_df,pd.DataFrame(results)],ignore_index=True)
log_df.to_csv(log_path,index=False)

best_entry=max(results,key=lambda x: x['mean_cv_r2'])
print(f"Best learning rate: {best_entry['learning_rate']} with mean R2={best_entry['mean_cv_r2']}")
print(log_df.tail())


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[458]	valid_0's rmse: 0.524316
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[285]	valid_0's rmse: 0.452472
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[699]	valid_0's rmse: 0.656654
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[238]	valid_0's rmse: 0.522455
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[136]	valid_0's rmse: 0.450373
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[634]	valid_0's rmse: 0.64237
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[157]	valid_0's rmse: 0.508741
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[70

CONCLUSION:
DONT USE:

##     'min_child_samples': 14,
##     'bagging_fraction': 0.7054945561615795,
####    'bagging_freq': 6

In [ ]:
#NEW SUBMISSION
    ##'lambda_l1': 0.9213114405513384,
    ##'lambda_l2': 0.6520962952151522

**SUBMISSION DEPRECATED**

In [71]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.65,
    learning_rate=0.01,
    max_depth=10,
    n_estimators=700,
    num_leaves=60,
    lambda_l1=0.7358897454113226,
    lambda_l2=0.23465766293184155,
    verbosity=-1,
    random_state=1
)



##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.65,
    learning_rate=0.03,
    max_depth=8,
    n_estimators=700,
    num_leaves=32,
    lambda_l1= 0.9213114405513384,
    lambda_l2= 0.6520962952151522,
    verbosity=-1,
    random_state=202
)
#grid_search_parameters(with cross fold)-parameters copied: --used for deprecated
#Final : 0.628903          Submitted - Submission 9
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.65,
##    'learning_rate': 0.03,
##    'max_depth': 8,
##    'n_estimators': 700,
##    'num_leaves': 32,
##    'verbosity': -1,
##    'random_state': 1,
##    'lambda_l1': 0.9213114405513384,
##    'lambda_l2': 0.6520962952151522
##}


##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_enhanced_with_seed_l1_tuned.csv',index=False)
print("Submission saved bro.csv")

Submission saved bro.csv


NORMAL_GRIDSEARCH_vs_SUBMISSION7

**SUBMISSION_BEST_OF_ALL**

In [83]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor( ##not changed yet
    objective='regression',
    metric='rmse',
    feature_fraction=0.5701089255338042,
    learning_rate=0.03346736377247299,
    max_depth=10,
    n_estimators=800,
    num_leaves=54,
    lambda_l1=0.8458897454113226,
    lambda_l2=0.43465766293184155,
    verbosity=-1,
    random_state=1
)
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.5701089255338042,
##    'learning_rate': 0.03346736377247299,
##    'max_depth': 10,
##    'n_estimators': 800,
##    'num_leaves': 54,
##    'lambda_l1': 0.8458897454113226,#53 - #84
##    'lambda_l2': 0.43465766293184155,#43
##    'verbosity': -1,
##    'random_state': 1
##}

##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2 with updated params
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.5,
    learning_rate=0.05,
    max_depth=7,
    n_estimators=800,
    num_leaves=47,
    lambda_l1= 0.9213114405513384,
    lambda_l2= 0.4520962952151522,
    random_state=1,
    verbosity =-1,
)

##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.5,
##    'learning_rate': 0.05,
##    'max_depth': 7,
##    'n_estimators': 800,
##    'num_leaves': 47,
##    'verbosity': -1,
##    'random_state': 1,
##    'lambda_l1': 0.9213114405513384,#92
##    'lambda_l2': 0.4520962952151522  #45
##}



##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_best_of_all.csv',index=False)
print("Submission saved bro.csv")

Submission saved bro.csv


**OVERALL_BEST_SUBMISSION**

In [56]:
import lightgbm as lgb
import pandas as pd

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combine all features for training and test
X_training_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y_training_full_Y1=training['Y1']
y_training_full_Y2=training['Y2']
X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

##defi model for Y1 with given best params
model_final_y1=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.45,
    learning_rate=0.01,
    max_depth=10,
    n_estimators=700,
    num_leaves=60,
    lambda_l1=0.7358897454113226,
    lambda_l2=0.23465766293184155,
    verbosity=-1,
    random_state=1
)



##training model for Y1 on full data
model_final_y1.fit(X_training_full,y_training_full_Y1)

##defi model for Y2 with updated params
model_final_y2=lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    feature_fraction=0.45,
    learning_rate=0.03,
    max_depth=9,
    n_estimators=700,
    num_leaves=92,
    lambda_l1= 0.9213114405513384,
    lambda_l2= 0.6520962952151522,
    verbosity=-1,
    random_state=202
)
#grid_search_parameters(with cross fold)-parameters copied: --used for deprecated
#Final : 0.628903          Submitted - Submission 9
##params={
##    'objective': 'regression',
##    'metric': 'rmse',
##    'feature_fraction': 0.55,
##    'learning_rate': 0.03,
##    'max_depth': 9,
##    'n_estimators': 700,#700
##    'num_leaves': 92,#32-#72
##    'verbosity': -1,
##    'random_state': 1,
##    'lambda_l1': 0.9213114405513384,
##    'lambda_l2': 0.6520962952151522
##}


##training model for Y2 on full data
model_final_y2.fit(X_training_full,y_training_full_Y2)

##predicting on test data
preds_y1_test=model_final_y1.predicting(X_test_full)
preds_y2_test=model_final_y2.predicting(X_test_full)

##preparing the submission DataFrame
submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_enhanced_22222.csv',index=False)
print("Submission saved bro.csv")

Submission saved bro.csv


*EARLY STOPPING*

In [14]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import training_test_split

features_base=list("ABCDEFGHIJKLMN")+['time']
new_features=['O','P']

##combining the  features given
X_full=pd.concat([training[features_base],training_new[new_features]],axis=1)
y1_full=training['Y1']
y2_full=training['Y2']

##Create validation splits (e.g.,80% training,20% val) to monitor overfitting
X_training1,X_val1,y_training1,y_val1=training_test_split(X_full,y1_full,test_size=0.2,random_state=1)
X_training2,X_val2,y_training2,y_val2=training_test_split(X_full,y2_full,test_size=0.2,random_state=202)

##Model params for Y1
params_y1={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.65,
    'learning_rate': 0.01,
    'max_depth': 10,
    'num_leaves': 60,
    'lambda_l1': 0.7358897454113226,
    'lambda_l2': 0.23465766293184155,
    'verbosity': -1,
    'random_state': 1
}

model_y1=lgb.LGBMRegressor(**params_y1,n_estimators=10000)

##training with early stopping on validation set
model_y1.fit(
    X_training1,y_training1,
    eval_set=[(X_val1,y_val1)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50)],
    #verbose=100
)



##Repeat for Y2
params_y2={
    'objective': 'regression',
    'metric': 'rmse',
    'feature_fraction': 0.65,
    'learning_rate': 0.03,
    'max_depth': 8,
    'num_leaves': 32,
    'lambda_l1': 0.9213114405513384,
    'lambda_l2': 0.6520962952151522,
    'verbosity': -1,
    'random_state': 202
}

model_y2=lgb.LGBMRegressor(**params_y2,n_estimators=10000)

model_y2.fit(
    X_training2,y_training2,
    eval_set=[(X_val2,y_val2)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=50)],
    #verbose=100
)

##After traininging with early stopping,use best iteration to predicting on test

X_test_full=pd.concat([test[features_base],test_new[new_features]],axis=1)

preds_y1_test=model_y1.predicting(X_test_full,num_iteration=model_y1.best_iteration_)
preds_y2_test=model_y2.predicting(X_test_full,num_iteration=model_y2.best_iteration_)

submission_df=pd.DataFrame({
    'id': test['id'].astype(int),
    'Y1': preds_y1_test.astype(float),
    'Y2': preds_y2_test.astype(float)
})

submission_df.to_csv('submission_with_early_stopping.csv',index=False)
print("Submission saved with early stopping")

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[669]	valid_0's rmse: 0.453846
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[412]	valid_0's rmse: 0.401177
Submission saved with early stopping


In [15]:
benchmark=pd.read_csv('/content/submission_enhanced_with_seed_l1_tuned.csv')

In [17]:
from sklearn.metrics import r2_score

##Ensure both dataframes are sorted by 'id' before calculating R2
submission_df_sorted=submission_df.sort_values(by='id').reset_index(drop=True)
benchmark_sorted=benchmark.sort_values(by='id').reset_index(drop=True)

##Calculate R2 for Y1 and Y2
r2_y1=r2_score(benchmark_sorted['Y1'],submission_df_sorted['Y1'])
r2_y2=r2_score(benchmark_sorted['Y2'],submission_df_sorted['Y2'])

print(f"R2 score for Y1: {r2_y1}")
print(f"R2 score for Y2: {r2_y2}")

##Calculate and print the average R2 score
average_r2=(r2_y1+r2_y2) / 2
print(f"Average R2 score: {average_r2}")

R2 score for Y1: 0.9967980151253079
R2 score for Y2: 0.9753096242689828
Average R2 score: 0.9860538196971453


In [18]:
best2=pd.read_csv('/content/submission_deprecated.csv')

In [19]:
from sklearn.metrics import r2_score

##Ensure both dataframes are sorted by 'id' before calculating R2
best2_sorted=best2.sort_values(by='id').reset_index(drop=True)
benchmark_sorted=benchmark.sort_values(by='id').reset_index(drop=True)

##Calculate R2 for Y1 and Y2
r2_y1_best2=r2_score(benchmark_sorted['Y1'],best2_sorted['Y1'])
r2_y2_best2=r2_score(benchmark_sorted['Y2'],best2_sorted['Y2'])

print(f"R2 score for Y1 (best2 vs benchmark): {r2_y1_best2}")
print(f"R2 score for Y2 (best2 vs benchmark): {r2_y2_best2}")

##Calculate and print the average R2 score
average_r2_best2=(r2_y1_best2+r2_y2_best2) / 2
print(f"Average R2 score (best2 vs benchmark): {average_r2_best2}")

R2 score for Y1 (best2 vs benchmark): 0.9993152644993969
R2 score for Y2 (best2 vs benchmark): 1.0
Average R2 score (best2 vs benchmark): 0.9996576322496984
